In [1]:
import json
import gc
import random
import networkx as nx
from networkx.readwrite import json_graph
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import scml
import mylib

In [2]:
# number of context words in the left and right directions
m = 2
iters = 1
type2id = {
    "click": 0,
    "cart": 1,
    "order": 2,
}

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(31)

In [4]:
%%time
with open("input/vocab2.json") as f:
    id2label = [w for w, c in json.load(f)]
label2id = {k:v for v, k in enumerate(id2label)}
print(f"len(label2id)={len(label2id):,}")

len(label2id)=1,855,603
Wall time: 2.21 s


In [5]:
del id2label
gc.collect()

21

In [6]:
%%time
with open("input/graph.json") as f:
    g = nx.adjacency_graph(json.load(f))
print(g)

DiGraph with 3747723 nodes and 99004731 edges
Wall time: 8min 20s


In [7]:
rows = []
for i in range(iters):
    for center in tqdm(g.nodes, desc=f"i={i}"):
        # going left: predecessor
        center_word, center_type = tuple(center.split("_"))
        curr = center
        for _ in range(m):
            choices = []
            for u, v, d in g.in_edges(curr, data=True):
                #print(f"u={u}, v={v}, d={d}")
                choices.append((u, d["weight"]))
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            curr_word, curr_type = tuple(curr.split("_"))
            rows.append({
                "center_word": label2id[center_word],
                "center_type": type2id[center_type],
                "outside_word": label2id[curr_word],
                "outside_type": type2id[curr_type],
            })
        # going right: successor
        curr = center
        for _ in range(m):
            choices = []
            for u, v, d in g.out_edges(curr, data=True):
                choices.append((v, d["weight"]))
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            curr_word, curr_type = tuple(curr.split("_"))
            rows.append({
                "center_word": label2id[center_word],
                "center_type": type2id[center_type],
                "outside_word": label2id[curr_word],
                "outside_type": type2id[curr_type],
            })

i=0: 100%|██████████████████| 3747723/3747723 [2:25:51<00:00, 428.25it/s]


In [8]:
df = pd.DataFrame.from_records(rows)
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
cols = ["center_word", "outside_word"]
df[cols] = df[cols].astype(np.int32)
cols = ["center_type", "outside_type"]
df[cols] = df[cols].astype(np.int8)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14463069 entries, 0 to 14463068
Data columns (total 4 columns):
 #   Column        Dtype
---  ------        -----
 0   center_word   int32
 1   center_type   int8 
 2   outside_word  int32
 3   outside_type  int8 
dtypes: int32(2), int8(2)
memory usage: 137.9 MB


In [9]:
df.describe(percentiles=percentiles)

,center_word,center_type,outside_word,outside_type
count,1.446307e+07,1.446307e+07,1.446307e+07,1.446307e+07
mean,7.882327e+05,6.641981e-01,4.145593e+05,2.612125e-01
std,5.170247e+05,7.512680e-01,4.613624e+05,6.006757e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,1.282300e+04,0.000000e+00,1.630000e+02,0.000000e+00
5%,6.438600e+04,0.000000e+00,2.249000e+03,0.000000e+00
10%,1.299880e+05,0.000000e+00,8.006000e+03,0.000000e+00
20%,2.658210e+05,0.000000e+00,3.016800e+04,0.000000e+00
30%,4.097160e+05,0.000000e+00,6.970300e+04,0.000000e+00
40%,5.635480e+05,0.000000e+00,1.314140e+05,0.000000e+00


In [10]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/pairs.parquet", index=False)

Wall time: 1.05 s


In [11]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 2:34:40.016259
